In [192]:
import sys
sys.path.append("..")
from lib.util import add_race_eth,  add_income_group
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1550
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [193]:
add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]

## All Lenders Combined
- **Missing: Minority Status, Tract/BNA**

In [194]:
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()
print("total number of loans",totalcount)
print("total amount of loans",totalamount)

('total number of loans', 242783)
('total amount of loans', 82066044)


In [197]:
### By Loan Purpose and Type

df['loan_purtype'] = '_'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] == 1)] = 'Purchase - Conventional'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] > 1)] = 'Purchase - Goverment'
df['loan_purtype'][df['loan_purpose'] == 2] = 'Home Improvement'
df['loan_purtype'][df['loan_purpose'] == 3] = 'Refinancing'

loanpt = df.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df['applicant_income_percent_median'] = 100*df.applicant_income/df.applicant_income.median()
df['applicant_income_cat'] = 'Income Not Available'
df['applicant_income_cat'][df['applicant_income_percent_median'] < 50] = 'Low'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 50) & (df['applicant_income_percent_median'] < 80)] = 'Moderate'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 80) & (df['applicant_income_percent_median'] < 120)] = 'Middle'
df['applicant_income_cat'][df['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications (1)'])
totalapp

Total Applications (1)          \
                                                                count  %count   
Loan Purpose and Type Home Improvement                           4781     2.0   
                      Purchase - Conventional                   32857    13.5   
                      Purchase - Goverment                      22240     9.2   
                      Refinancing                              182905    75.3   
Applicant Race/Eth    aian                                        767     0.3   
                      as                                        23920     9.9   
                      b                                          4053     1.7   
                      h                                         24012     9.9   
                      pi                                         1908     0.8   
                      w                                        142000    58.5   
Applicant Income      Income Not Available                      28473    11.7   
                      Low                                       26217    10.8   
                      Middle                                    57469    23.7   
                      Moderate                                  49857    20.5   
                      Upper                                     80767    33.3   
Applicant Sex         1                                        159485    65.7   
                      2                                         50394    20.8   
                      3                                         17527     7.2   
                      4                                         15377     6.3   

                                                                  
                                                 amount  %amount  
Loan Purpose and Type Home Improvement          1038687      1.3  
                      Purchase - Conventional  12419787     15.1  
                      Purchase - Goverment      7551607      9.2  
                      Refinancing              61055963     74.4  
Applicant Race/Eth    aian                       231319      0.3  
                      as                        7808190      9.5  
                      b                         1139822      1.4  
                      h                         6385159      7.8  
                      pi                         570463      0.7  
                      w                        48384565     59.0  
Applicant Income      Income Not Available     11876933     14.5  
                      Low                       4777981      5.8  
                      Middle                   17701587     21.6  
                      Moderate                 12375557     15.1  
                      Upper                    35333986     43.1  
Applicant Sex         1                        54930806     66.9  
                      2                        14172171     17.3  
                      3                         5939329      7.2  
                      4                         7023738      8.6

Action Taken:
* 1 -- Loan originated
* 2 -- Application approved but not accepted
* 3 -- Application denied by financial institution
* 4 -- Application withdrawn by applicant
* 5 -- File closed for incompleteness
* 6 -- Loan purchased by the institution
* 7 -- Preapproval request denied by financial institution
* 8 -- Preapproval request approved but not accepted (optional reporting)

In [169]:
df.action_type.unique()

array([1, 2, 3, 4, 6, 5])

In [198]:
####################
# ACTION TYPE == 1
####################
df1 = df[df.action_type==1]

### By Loan Purpose and Type

df1['loan_purtype'] = '_'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] == 1)] = 'Purchase - Conventional'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] > 1)] = 'Purchase - Goverment'
df1['loan_purtype'][df1['loan_purpose'] == 2] = 'Home Improvement'
df1['loan_purtype'][df1['loan_purpose'] == 3] = 'Refinancing'

loanpt = df1.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df1.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df1['applicant_income_percent_median'] = 100*df1.applicant_income/df1.applicant_income.median()
df1['applicant_income_cat'] = 'Income Not Available'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] < 50] = 'Low'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 50) & (df1['applicant_income_percent_median'] < 80)] = 'Moderate'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 80) & (df1['applicant_income_percent_median'] < 120)] = 'Middle'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df1.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df1.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp1 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp1.ix[:, ["count","%count","amount","%amount",]]]
totalapp1 = pd.concat(pieces, axis=1, keys=['Action Type = 1'])
totalapp1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:35: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:36: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Action Type = 1          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                    2315     1.0   
                      Purchase - Conventional            18452     7.6   
                      Purchase - Goverment               10545     4.3   
                      Refinancing                       103541    42.6   
Applicant Race/Eth    aian                                 417     0.2   
                      as                                 14274     5.9   
                      b                                   2219     0.9   
                      h                                  12799     5.3   
                      pi                                  1027     0.4   
                      w                                  86821    35.8   
Applicant Income      Income Not Available               12748     5.3   
                      Low                                14379     5.9   
                      Middle                             33853    13.9   
                      Moderate                           28098    11.6   
                      Upper                              45775    18.9   
Applicant Sex         1                                  95760    39.4   
                      2                                  29277    12.1   
                      3                                   8844     3.6   
                      4                                    972     0.4   

                                                                  
                                                 amount  %amount  
Loan Purpose and Type Home Improvement           624517      0.8  
                      Purchase - Conventional   7434158      9.1  
                      Purchase - Goverment      3629798      4.4  
                      Refinancing              34805947     42.4  
Applicant Race/Eth    aian                       127627      0.2  
                      as                        4702467      5.7  
                      b                          630383      0.8  
                      h                         3417863      4.2  
                      pi                         309668      0.4  
                      w                        29334688     35.7  
Applicant Income      Income Not Available      6322209      7.7  
                      Low                       2606371      3.2  
                      Middle                   10424061     12.7  
                      Moderate                  6996014      8.5  
                      Upper                    20145765     24.5  
Applicant Sex         1                        32921809     40.1  
                      2                         8218736     10.0  
                      3                         3100921      3.8  
                      4                         2252954      2.7

In [199]:

#######
# Action Type =2 
#######

df2 = df[df.action_type==2]

### By Loan Purpose and Type

df2['loan_purtype'] = '_'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] == 1)] = 'Purchase - Conventional'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] > 1)] = 'Purchase - Goverment'
df2['loan_purtype'][df2['loan_purpose'] == 2] = 'Home Improvement'
df2['loan_purtype'][df2['loan_purpose'] == 3] = 'Refinancing'

loanpt = df2.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df2.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df2['applicant_income_percent_median'] = 100*df2.applicant_income/df2.applicant_income.median()
df2['applicant_income_cat'] = 'Income Not Available'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] < 50] = 'Low'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 50) & (df2['applicant_income_percent_median'] < 80)] = 'Moderate'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 80) & (df2['applicant_income_percent_median'] < 120)] = 'Middle'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df2.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df2.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp2 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp2.ix[:, ["count","%count","amount","%amount",]]]
totalapp2 = pd.concat(pieces, axis=1, keys=['Action Type = 2'])
totalapp2

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Action Type = 2          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                     221     0.1   
                      Purchase - Conventional             1918     0.8   
                      Purchase - Goverment                 747     0.3   
                      Refinancing                         6872     2.8   
Applicant Race/Eth    aian                                  40     0.0   
                      as                                  1185     0.5   
                      b                                    178     0.1   
                      h                                   1114     0.5   
                      pi                                    71     0.0   
                      w                                   5841     2.4   
Applicant Income      Income Not Available                 699     0.3   
                      Low                                 1157     0.5   
                      Middle                              2403     1.0   
                      Moderate                            2009     0.8   
                      Upper                               3490     1.4   
Applicant Sex         1                                   6732     2.8   
                      2                                   2229     0.9   
                      3                                    765     0.3   
                      4                                     32     0.0   

                                                                 
                                                amount  %amount  
Loan Purpose and Type Home Improvement           31193      0.0  
                      Purchase - Conventional   656007      0.8  
                      Purchase - Goverment      245548      0.3  
                      Refinancing              2255775      2.7  
Applicant Race/Eth    aian                       12336      0.0  
                      as                        380775      0.5  
                      b                          46575      0.1  
                      h                         286285      0.3  
                      pi                         21871      0.0  
                      w                        1955672      2.4  
Applicant Income      Income Not Available      264188      0.3  
                      Low                       178175      0.2  
                      Middle                    717774      0.9  
                      Moderate                  465712      0.6  
                      Upper                    1562674      1.9  
Applicant Sex         1                        2292781      2.8  
                      2                         594259      0.7  
                      3                         262191      0.3  
                      4                          39292      0.0

In [203]:
#Merging ALL
totalapp = pd.merge(totalapp, totalapp1, totalapp2)

MergeError: No common columns to perform merge on

#By Banks

In [165]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = source[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('BANK OF AMERICA')][['respondent_id','respondent_name']].drop_duplicates()

NameError: name 'source' is not defined

# Bank of America
- respondent_id = 0000480228 

In [172]:
boatotalcount = boadf.loan_amount.count()
boatotalamount = boadf.loan_amount.sum()

# classification
boadf['loan_purtype'] = '_'
boadf['loan_purtype'][(boadf['loan_purpose'] == 1) & (boadf['loan_type'] == 1)] = 'Purchase - Conventional'
boadf['loan_purtype'][(boadf['loan_purpose'] == 1) & (boadf['loan_type'] > 1)] = 'Purchase - Goverment'
boadf['loan_purtype'][boadf['loan_purpose'] == 2] = 'Home Improvement'
boadf['loan_purtype'][boadf['loan_purpose'] == 3] = 'Refinancing'

loanpt = boadf.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/boatotalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/boatotalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

race_eth  = boadf.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/boatotalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/boatotalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

# classification of applicant income as percent of median
boadf['applicant_income_percent_median'] = 100*boadf.applicant_income/boadf.applicant_income.median()
boadf['applicant_income_cat'] = 'Income Not Available'
boadf['applicant_income_cat'][boadf['applicant_income_percent_median'] < 50] = 'Low'
boadf['applicant_income_cat'][(boadf['applicant_income_percent_median'] >= 50) & (boadf['applicant_income_percent_median'] < 80)] = 'Moderate'
boadf['applicant_income_cat'][(boadf['applicant_income_percent_median'] >= 80) & (boadf['applicant_income_percent_median'] < 120)] = 'Middle'
boadf['applicant_income_cat'][boadf['applicant_income_percent_median'] >= 120] = 'Upper'

income  = boadf.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/boatotalcount,1)
income ['%amount'] = np.round(100*income['amount']/boatotalamount,1)
income = income [['count','%count','amount','%amount']]

sex  = boadf.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/boatotalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/boatotalamount,1)
sex = sex [['count','%count','amount','%amount']]

frames = [loanpt, race_eth,income,sex]
boaapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])

pieces = [boaapp.ix[:, ["count","%count","amount","%amount",]]]
boaapp = pd.concat(pieces, axis=1, keys=['Total Applications(1)'])
boaapp

Total Applications(1)          \
                                                               count  %count   
Loan Purpose and Type Home Improvement                           103     1.0   
                      Purchase - Conventional                   1022    10.3   
                      Purchase - Goverment                       550     5.5   
                      Refinancing                               8271    83.2   
Applicant Race/Eth    aian                                        40     0.4   
                      as                                        1023    10.3   
                      b                                          201     2.0   
                      h                                         1463    14.7   
                      pi                                          81     0.8   
                      w                                         5668    57.0   
Applicant Income      Income Not Available                      1462    14.7   
                      Low                                       1495    15.0   
                      Middle                                    1699    17.1   
                      Moderate                                  1828    18.4   
                      Upper                                     3462    34.8   
Applicant Sex         1                                         6979    70.2   
                      2                                         2324    23.4   
                      3                                          620     6.2   
                      4                                           23     0.2   

                                                                 
                                                amount  %amount  
Loan Purpose and Type Home Improvement           32417      0.9  
                      Purchase - Conventional   484014     13.9  
                      Purchase - Goverment      174186      5.0  
                      Refinancing              2783718     80.1  
Applicant Race/Eth    aian                       11999      0.3  
                      as                        329398      9.5  
                      b                          56321      1.6  
                      h                         354973     10.2  
                      pi                         22402      0.6  
                      w                        2052423     59.1  
Applicant Income      Income Not Available      504627     14.5  
                      Low                       275911      7.9  
                      Middle                    484262     13.9  
                      Moderate                  419810     12.1  
                      Upper                    1789725     51.5  
Applicant Sex         1                        2524987     72.7  
                      2                         645036     18.6  
                      3                         255241      7.3  
                      4                          49071      1.4

In [167]:
#merging
pd.merge(totalapp, boaapp, left_index=True, right_index=True)

Total Applications          \
                                                            count  %count   
Loan Purpose and Type Home Improvement                       4781     2.0   
                      Purchase - Conventional               32857    13.5   
                      Purchase - Goverment                  22240     9.2   
                      Refinancing                          182905    75.3   
Applicant Race/Eth    aian                                    767     0.3   
                      as                                    23920     9.9   
                      b                                      4053     1.7   
                      h                                     24012     9.9   
                      pi                                     1908     0.8   
                      w                                    142000    58.5   
Applicant Income      Income Not Available                  28473    11.7   
                      Low                                   26217    10.8   
                      Middle                                57469    23.7   
                      Moderate                              49857    20.5   
                      Upper                                 80767    33.3   
Applicant Sex         1                                    159485    65.7   
                      2                                     50394    20.8   
                      3                                     17527     7.2   
                      4                                     15377     6.3   

                                                                  \
                                                 amount  %amount   
Loan Purpose and Type Home Improvement          1038687      1.3   
                      Purchase - Conventional  12419787     15.1   
                      Purchase - Goverment      7551607      9.2   
                      Refinancing              61055963     74.4   
Applicant Race/Eth    aian                       231319      0.3   
                      as                        7808190      9.5   
                      b                         1139822      1.4   
                      h                         6385159      7.8   
                      pi                         570463      0.7   
                      w                        48384565     59.0   
Applicant Income      Income Not Available     11876933     14.5   
                      Low                       4777981      5.8   
                      Middle                   17701587     21.6   
                      Moderate                 12375557     15.1   
                      Upper                    35333986     43.1   
Applicant Sex         1                        54930806     66.9   
                      2                        14172171     17.3   
                      3                         5939329      7.2   
                      4                         7023738      8.6   

                                               Bank of America          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                     103     1.0   
                      Purchase - Conventional             1022    10.3   
                      Purchase - Goverment                 550     5.5   
                      Refinancing                         8271    83.2   
Applicant Race/Eth    aian                                  40     0.4   
                      as                                  1023    10.3   
                      b                                    201     2.0   
                      h                                   1463    14.7   
                      pi                                    81     0.8   
                      w                                   5668    57.0   
Applicant Income      Income Not Available                1462    14.7   
                      Low             